In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim

/Users/mng/miniforge3/envs/c3-nlp/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/mng/miniforge3/envs/c3-nlp/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowExxb
  Referenced from: /Users/mng/miniforge3/envs/c3-nlp/lib/python3.8/site-packages/torchvision/image.so
  Expected in: /Users/mng/miniforge3/envs/c3-nlp/lib/python3.8/site-packages/torch/lib/libc10.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
batch_size = 64
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081))])

In [3]:
train_dataset = datasets.MNIST(root='../dataset/mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

train_dataset = datasets.MNIST(root='../dataset/mnist/', train=False, download=True, transform=transform)
test_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw



In [4]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.linear1 = torch.nn.Linear(784, 512)
    self.linear2 = torch.nn.Linear(512, 256)
    self.linear3 = torch.nn.Linear(256, 128)
    self.linear4 = torch.nn.Linear(128, 64)
    self.linear5 = torch.nn.Linear(64, 10)

  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.linear1(x))
    x = F.relu(self.linear2(x))
    x = F.relu(self.linear3(x))
    x = F.relu(self.linear4(x))
    x = self.linear5(x)
    return x

model = Net()

In [5]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [6]:
def train(epoch):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    inputs, target = data
    optimizer.zero_grad()

    output = model(inputs)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

    if i % 300 == 299:
      print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 300))
      running_loss = 0.0

In [ ]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_loader:
      images, labels = data
      outputs = model(images)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

  print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))